In [1]:
__PRODUCTION__ = 1
__NAME__       = 'efficiency'
__WIDTH__      = 5.5  # NeurIPS 2021 text box width
__HEIGHT__     = 1.85

if __PRODUCTION__:
    from mplmagic2 import pgf
else:
    from mplmagic2 import svg

from mplmagic2 import SuperFigure, tex
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.patches import Rectangle

print('This is how much space the figure will take up on letter paper')
SuperFigure.size_hint(__WIDTH__, __HEIGHT__, margin_left=0.5 * (8.5 - __WIDTH__));

This is how much space the figure will take up on letter paper


In [2]:
import functools
import numpy as np
import colorsys
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F
from torch import optim
import networkx as nx
import json
import pickle

from symfac.experimental import RBFExpansionV2

In [3]:
rgn_losses = pickle.loads(open('data/paper/rgn_losses.pickle', 'rb').read())
gn_losses = pickle.loads(open('data/paper/gn_losses.pickle', 'rb').read())
er_losses = pickle.loads(open('data/paper/er_losses.pickle', 'rb').read())
ba_losses = pickle.loads(open('data/paper/ba_losses.pickle', 'rb').read())

In [4]:
data = {
    'rectangular-gaussian-noise': {
        'rbf': np.array(rgn_losses[0]),
        'svd': np.array(rgn_losses[1]),
    },
    'square-gaussian-noise': {
        'rbf': np.array(gn_losses[0]),
        'svd': np.array(gn_losses[1]),
    },
    'erdos-renyi': {
        'svd': np.array(er_losses[1]),
        'rbf': np.array(er_losses[0]),
    },
      'barabasi-albert': {
        'svd': np.array(ba_losses[1]),
        'rbf': np.array(ba_losses[0]),
    },
}

# open('data/efficiency.json', 'w').write(json.dumps(data))

In [7]:
fig = SuperFigure(figsize=(__WIDTH__, __HEIGHT__), dpi=300)
ax_canvas = fig.make_axes(
    left=0, right=1, top=0, bottom=1, zorder=-100,
    style='blank' if __PRODUCTION__ else None
)
ax_canvas.set_xlim([0, 1])
ax_canvas.set_ylim([0, 1])

# draw the axes grid
x0 = 0.055
dx = 0.235
dxx = 0.015
w = 0.22
axs = [
    fig.make_axes(
        left=x0 + dx * i + (dxx if i > 0 else 0),
        width=w,
        top=0.12,
        width_to_height=1.0,
        style='modern'
    ) for i in range(4)
]


def draw_bars(ax, h, offset, **kwargs):
    ax.bar(np.arange(len(h)) + offset, h, **kwargs)

bar_width = 0.425
bar_style = dict(
    width=bar_width
)
svd_style = dict(
    color='#30BB60',
    label=r'\textbf{SVD}'
)
rbf_style = dict(
    color='k',
    label=r'\textbf{RBF}'
)
tick_style = dict(
    fontsize=6
)

for i, (ax, (key, case)) in enumerate(zip(axs, data.items())):
    ax.set_yscale('log')
    draw_bars(ax, case['svd'], -0.5 * bar_width, **bar_style, **svd_style)
    draw_bars(ax, case['rbf'], 0.5 * bar_width, **bar_style, **rbf_style)
    n = np.max([len(val) for val in case.values()])
    ax.set_xbound(lower=0, upper=n)
    xticks = np.arange(1, n + 1, (n - 9) // 10 + 1)
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticks, fontsize=6)
    ymin, ymax = max(1e-6, case['svd'].min()), 1
    ax.set_ybound(lower=ymin, upper=ymax)
    ax.set_ylim([ymin, ymax])
    yticks = np.linspace(np.log10(ymin), np.log10(ymax), 3)
    ax.set_yticks(10**yticks)
    if i == 0:
        ax.set_yticklabels([fr'${tex.pow(10, int(y))}$' for y in yticks], **tick_style)
    else:
        ax.set_yticklabels([])
    ax.tick_params(axis='both', which='both', length=1.5, direction='out')

    if i == 1:
        extra_ticks = np.linspace(-1, -3, 3)
        all_ticks = np.concatenate((yticks, extra_ticks))
        all_ticklabels = np.array([''] * len(yticks) + [fr'${tex.pow(10, int(y))}$' for y in extra_ticks])
        ax.set_yticks(10**all_ticks)
        ax.set_yticklabels(all_ticklabels, **tick_style)
        ax.tick_params(
            axis='y', which='both', length=1, direction='out', pad=0
        )
        ax.set_ybound(lower=ymin, upper=ymax)
        ax.set_ylim([ymin, ymax])
        for j, ref in enumerate(10**extra_ticks):
            r_rbf, r_svd = (
                np.flatnonzero(case['rbf'] <= ref)[0] + 1,
                np.flatnonzero(case['svd'] <= ref)[0] + 1
            )
            r_max = len(case['svd'])
            ax.axhline(
                ref,
                color='w',
                lw=0.5,
                ls=(2, (1.7, 3.1)),
                dash_capstyle='round',
                zorder=30
            )
            ax.text(
                r_max, ref * 0.8, f'RBF {r_rbf} / {r_svd} SVD  ',
                ha='right',
                va='top',
                fontsize=7,
                zorder=20,
                path_effects=[
                    path_effects.Stroke(linewidth=1.5, foreground='w'),
                    path_effects.Normal()
                ]        
            )
            patch_w = 32
            ax.add_patch(
                Rectangle(
                    (r_max - patch_w, ref * 0.25), patch_w, 0.75 * ref,
                    color='w',
                    alpha=0.9,
                    zorder=10
                ),
            )

label_style = dict(
    fontsize=8,
)
for i, ax in enumerate(axs):
    if i == 0:
        ax.set_ylabel(r'MSE loss', labelpad=1, **label_style)
    ax.set_xlabel(r'components', labelpad=3, **label_style)
    
title_style = dict(
    fontsize=8,
    y=0.975,
    va='bottom'
)
axs[0].set_title(r'\textbf{Gaussian Noise} 16$\times$64', **title_style)
axs[1].set_title(r'\textbf{Gaussian Noise} 64$\times$64', **title_style)
axs[2].set_title(r'\textbf{Erdos-Renyi} 40$\times$40', **title_style)
axs[3].set_title(r'\textbf{Barabasi–Albert} 40$\times$40', **title_style)

panel_number_style = dict(
    fontsize=8,
    va='top',
    ha='right'
)
for i, ax in enumerate(axs):
    ax.text(
        0.975, 0.975,
        s=fr'\textbf{{({chr(65 + i)})}}',
        **panel_number_style,
        transform=ax.transAxes,
    )

ax_canvas.bar([-1], [0], **bar_style, **svd_style)
ax_canvas.bar([-1], [0], **bar_style, **rbf_style)
ax_canvas.legend(
    loc='center',
    bbox_to_anchor=(0.5, 0.03),
    ncol=2,
    frameon=False,
    fontsize=6
)


if __PRODUCTION__:
    fig.savefig(f'pgf/{__NAME__}.pgf', dpi=300)
else:
    fig.savefig(f'svg/{__NAME__}.svg', dpi=300)
plt.show()

In [8]:
!make -f Makefile.figures fig-"$__NAME__".pdf 2>&1 | tail -n 1

Successfully created fig-efficiency.pdf


END
---